<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/query_engine/knowledge_graph_query_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 패키지 설치

In [ ]:
%pip install -q llama-index-graph-stores-neo4j
%pip install -q llama-index-llms-openai llama-index
%pip install -q ipython-ngql nebula3-python python-dotenv

### NebulaGraph console

- llamaindex space 만들기

- TextNode 구조


    text=entry["content"],

    metadata={
                    "law_index": entry["index"],
                    "subtitle": entry["subtitle"],
                    "document_title": entry["metadata"]["document_title"],
                    "created_date": entry["metadata"]["date"],
                    "revise_info": entry["metadata"]["revise_info"],
                    "source": entry["metadata"]["source"],
                    "title_doc": entry["metadata"]["title"]["doc"],
                    "title_chapter": entry["metadata"]["title"]["chapter"],
                    "title_section": entry["metadata"]["title"]["section"],
                    "title_subsection": entry["metadata"]["title"]["subsection"],
                }

In [2]:
# 터미널에서 실행
# cd /mnt/c/Users/Shic/legal_graph/nebula-console
# ./nebula-console -addr=192.168.176.1 -port 9669 -u root -p 001101

# If not, create it with the following commands from NebulaGraph's console:
# CREATE SPACE llamaindex(vid_type=FIXED_STRING(2048), partition_num=1, replica_factor=1);
# USE llamaindex;
# CREATE TAG law(law_index string, name string, document_title string, created_date string, revise_info string, source string, title_doc string, title_chapter string, title_section string, title_subsection string);
# CREATE EDGE REFERS_TO(relation string);
# CREATE TAG INDEX law_index ON law(law_index(256));

> https://docs.nebula-graph.io/3.0.0/3.ngql-guide/9.space-statements/1.create-space/

> https://docs.nebula-graph.io/3.0.0/3.ngql-guide/10.tag-statements/1.create-tag/

### OpenAI

In [1]:
# For OpenAI

import os
from dotenv import load_dotenv
load_dotenv(verbose=True)

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output


In [4]:
CHUNK_SIZE = 1024
CHUNK_OVERLAP = 50

In [5]:
# define LLM
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-4o")
Settings.chunk_size = CHUNK_SIZE
Settings.chunk_overlap = CHUNK_OVERLAP

## Prepare for Neo4j graph

Before next step to creating the Knowledge Graph, let's ensure we have a running NebulaGraph with defined data schema.

In [2]:
from llama_index.core import KnowledgeGraphIndex
from llama_index.core import StorageContext
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from IPython.display import Markdown, display
from llama_index.core import PropertyGraphIndex

In [6]:
import os

import json
from llama_index.core import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

# JSON 파일을 로드하는 함수
def load_json_as_documents(input_file):
    # 파일을 열고 JSON 데이터를 파싱
    with open(input_file, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

    # # schema에 맞추어 json flatten
    # flattened_json = [flatten_json(item) for item in json_data]

    documents = []; doc_length = []
    
    # JSON 리스트의 각 요소를 Document로 변환하여 리스트에 추가
    for entry in json_data:
        # 각 요소를 Document 객체로 변환
        doc = Document(
            text=entry["content"],
            metadata={
                "law_index": entry["index"],
                "name": entry["subtitle"],
                "document_title": entry["metadata"]["document_title"],
                "created_date": entry["metadata"]["date"],
                "revise_info": entry["metadata"]["revise_info"],
                "source": entry["metadata"]["source"],
                "title_doc": entry["metadata"]["title"]["doc"],
                "title_chapter": entry["metadata"]["title"]["chapter"],
                "title_section": entry["metadata"]["title"]["section"],
                "title_subsection": entry["metadata"]["title"]["subsection"],
            },
            metadata_seperator="::",
            metadata_template="{key}=>{value}",
            text_template="Metadata: {metadata_str}\n\nText: {content}",
        )
        documents.append(doc)
        doc_length.append(len(doc.text))
    
    return documents, doc_length

# 텍스트를 분할하는 함수
def split_documents(documents, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    
    split_docs = []
    
    for doc in documents:
        # 텍스트를 분할하고 각 조각을 Document로 다시 생성
        chunks = text_splitter.split_text(doc.text)
        for chunk in chunks:
            split_doc = Document(
                text=chunk,
                metadata=doc.metadata,
                metadata_seperator=doc.metadata_seperator,
                metadata_template=doc.metadata_template,
                text_template=doc.text_template
            )
            split_docs.append(split_doc)
    
    return split_docs

# JSON 파일 경로
input_file = "/mnt/c/Users/Shic/legal_graph/results/1-2/DCM_1-2_law_main_clause.json"

# 로더 사용하여 데이터 불러오기
documents, doc_length = load_json_as_documents(input_file)

In [7]:
import numpy as np

np.mean(doc_length)

179.5015243902439

In [16]:
np.percentile(doc_length, [0, 25, 50, 75, 100])

array([  15.  ,   77.  ,  121.  ,  204.25, 4800.  ])

In [9]:
max(doc_length)

4800

In [13]:
min(doc_length)

15

In [ ]:
# 문서를 512 토큰 크기로 분할
documents = split_documents(documents)


In [ ]:
from neo4j import GraphDatabase

graph_store = Neo4jPropertyGraphStore(
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    url=os.getenv("NEO4J_URI"),
    database=os.getenv("NEO4J_DATABASE"),
)


load_dotenv(verbose=True)

# Neo4j 서버에 연결
uri = os.getenv("NEO4J_URI")
user = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")


# URI와 사용자 정보 설정
AUTH = (user, password)

# GraphDatabase에 드라이버 생성 및 연결 확인
with GraphDatabase.driver(uri, auth=(user, password)) as driver:
    driver.verify_connectivity()
    print("Neo4j 서버와의 연결이 성공적으로 확인되었습니다.")


In [ ]:
len(documents)

In [ ]:
documents[58]

In [ ]:
from llama_index.core.schema import MetadataMode
print(
    "The LLM sees this: \n",
    documents[58].get_content(metadata_mode=MetadataMode.LLM),
)

In [ ]:
list(documents[0].metadata.keys())

In [10]:
# # 예시 템플릿
# CUSTOM_KG_TRIPLET_EXTRACT_TMPL = (
#     "Extract up to {max_knowledge_triplets} knowledge triplets from the given text. "
#     "Each triplet should be in the form of (head, relation, tail) with their respective types.\n"
#     "---------------------\n"
#     "INITIAL ONTOLOGY:\n"
#     "Entity Types: {allowed_entity_types}\n"
#     "Relation Types: {allowed_relation_types}\n"
#     "\n"
#     "Use these types as a starting point, but introduce new types if necessary based on the context.\n"
#     "\n"
#     "GUIDELINES:\n"
#     "- Output in JSON format: [{{'head': '', 'head_type': '', 'relation': '', 'tail': '', 'tail_type': ''}}]\n"
#     "- Use the most complete form for entities (e.g., 'United States of America' instead of 'USA')\n"
#     "- Keep entities concise (3-5 words max)\n"
#     "- Break down complex phrases into multiple triplets\n"
#     "- Ensure the knowledge graph is coherent and easily understandable\n"
#     "---------------------\n"
#     "EXAMPLE:\n"
#     "Text: Tim Cook, CEO of Apple Inc., announced the new Apple Watch that monitors heart health. "
#     "UC Berkeley researchers studied the benefits of apples.\n"
#     "Output:\n"
#     "[{{'head': 'Tim Cook', 'head_type': 'PERSON', 'relation': 'CEO_OF', 'tail': 'Apple Inc.', 'tail_type': 'COMPANY'}},\n"
#     " {{'head': 'Apple Inc.', 'head_type': 'COMPANY', 'relation': 'PRODUCES', 'tail': 'Apple Watch', 'tail_type': 'PRODUCT'}},\n"
#     " {{'head': 'Apple Watch', 'head_type': 'PRODUCT', 'relation': 'MONITORS', 'tail': 'heart health', 'tail_type': 'HEALTH_METRIC'}},\n"
#     " {{'head': 'UC Berkeley', 'head_type': 'UNIVERSITY', 'relation': 'STUDIES', 'tail': 'benefits of apples', 'tail_type': 'RESEARCH_TOPIC'}}]\n"
#     "---------------------\n"
#     "Text: {text}\n"
#     "Output:\n"
# )

In [10]:
# from llama_index.core import Prompt

triplet_prompt = """
당신은 법률 지식을 가지고 있는 그래프 데이터베이스 구축 전문가입니다.
법률 그래프 데이터베이스에 노드와 엣지를 삽입하기 위해서, 아래 [지시사항]을 따라 text에서 ({{Subject entity}}, {{Relationship}}, {{Object Entity}}) 형태의 triplet을 추출해주세요.

[지시사항]
- 주어진 텍스트에서 가능한 모든 사실과 법 조항 간의 관계를 추출하세요.
- 특히, 법 조항 간의 참조, 의무, 금지 등의 관계를 중점적으로 추출하세요.
- 또한 문장에서 언급되는 특정 개체가 행하거나, 영향을 미치는 관계도 추출해주세요.

- 출력은 다음과 같은 형식: [({{'entity_type' : '', 'law_index': '', 'name': '', 'document_title': '', 'created_date': '', 'revise_info': '', 'source' : '', 'title_doc': '', 'title_chapter' : '', 'title_section' : '', 'title_subsection' : ''}}, 
                                {{'relation_type' : '', 'description' : ''}},
                                {{'entity_type' : '', 'law_index': '', 'name': '', 'document_title': '', 'created_date': '', 'revise_info': '', 'source' : '', 'title_doc': '', 'title_chapter' : '', 'title_section' : '', 'title_subsection' : ''}})]
- relation에 있는 description에는 **왜 해당 relation_type으로 지정되었는지"에 대한 근거를 작성해주세요.
- 복잡한 구문은 여러 개의 삼중항(triplet)으로 분리합니다.

각 entity_type이 의미하는 바는 다음과 같습니다.
- LAW : 법률
- HUMAN : 사람
- ACTION : 행동
- PENALTY : 벌칙, 벌금
- OBJECT : 그 외 개체

각 relation_type이 의미하는 바는 다음과 같습니다.
- CONNECT : S는 O와 연결관계가 있다
- DECIDE_OF : S는 O를 결정한다
- ACT : S가 O를 행하다
- REFERS_TO : S가 O를 참조하다
- CAUSE : S가 O를 야기하다
- EQUALS_TO : S는 O와 동일하다
- INCLUDES : S는 O를 포함한다
- EFFECT : S는 O에 영향을 미친다


[예시]
<input>
 Metadata: law_index=>제125조제1항::name=>거짓의 기재 등으로 인한 배상책임::document_title=>자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )
::created_date=>시행 2024. 8. 14.::revise_info=>법률 제20305호, 2024. 2. 13., 일부개정
::source=>국가법령정보센터::title_doc=>제3편 증권의 발행 및 유통::title_chapter=>제1장 증권신고서::title_section=>제2절 금융투자업자별 영업행위 규칙::title_subsection=>제4관 신탁업자의 영업행위 규칙

Text: 증권신고서(정정신고서 및 첨부서류를 포함한다. 이하 이 조에서 같다)와 투자설명서(예비투자설명서 및 간이투자설명서를 포함한다. 이하 이 조에서 같다) 중 중요사항에 관하여 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 증권의 취득자가 손해를 입은 경우에는 다음 각 호의 자는 그 손해에 관하여 배상의 책임을 진다. 다만, 배상의 책임을 질 자가 상당한 주의를 하였음에도 불구하고 이를 알 수 없었음을 증명하거나 그 증권의 취득자가 취득의 청약을 할 때에 그 사실을 안 경우에는 배상의 책임을 지지 아니한다. <개정 2009. 2. 3., 2013. 5. 28.>\n1. 그 증권신고서의 신고인과 신고 당시의 발행인의 이사(이사가 없는 경우 이에 준하는 자를 말하며, 법인의 설립 전에 신고된 경우에는 그 발기인을 말한다)\n2. 「상법」 제401조의2제1항 각 호의 어느 하나에 해당하는 자로서 그 증권신고서의 작성을 지시하거나 집행한 자\n3. 그 증권신고서의 기재사항 또는 그 첨부서류가 진실 또는 정확하다고 증명하여 서명한 공인회계사ㆍ감정인 또는 신용평가를 전문으로 하는 자 등(그 소속단체를 포함한다) 대통령령으로 정하는 자\n4. 그 증권신고서의 기재사항 또는 그 첨부서류에 자기의 평가ㆍ분석ㆍ확인 의견이 기재되는 것에 대하여 동의하고 그 기재내용을 확인한 자\n5. 그 증권의 인수인 또는 주선인(인수인 또는 주선인이 2인 이상인 경우에는 대통령령으로 정하는 자를 말한다)\n6. 그 투자설명서를 작성하거나 교부한 자\n7. 매출의 방법에 의한 경우 매출신고 당시의 매출인

<output>
[
({{'entity_type' : 'HUMAN', 'law_index': '제125조제1항', 'name': '증권신고서의 신고인과 신고 당시의 발행인의 이사', 'document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )', 'created_date': '시행 2024. 8. 14.', 'revise_info': '법률 제20305호, 2024. 2. 13., 일부개정', 'source' : '국가법령정보센터', 'title_doc': '제3편 증권의 발행 및 유통', 'title_chapter' : '제1장 증권신고서', 'title_section' : '제2절 금융투자업자별 영업행위 규칙', 'title_subsection' : '제4관 신탁업자의 영업행위 규칙'}}, 
{{'relation_type' : 'ACT', 'description' : '증권신고서의 신고인과 신고 당시의 발행인의 이사가 배상책임을 행함'}},
{{'entity_type' : 'PENALTY', 'law_index': '제125조제1항', 'name': '배상책임', 'document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )', 'created_date': '시행 2024. 8. 14.', 'revise_info': '법률 제20305호, 2024. 2. 13., 일부개정', 'source' : '국가법령정보센터', 'title_doc': '제3편 증권의 발행 및 유통', 'title_chapter' : '제1장 증권신고서', 'title_section' : '제2절 금융투자업자별 영업행위 규칙', 'title_subsection' : '제4관 신탁업자의 영업행위 규칙'}}),

({{'entity_type' : 'OBJECT', 'law_index': '제125조제1항', 'name': '증권신고서', 'document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )', 'created_date': '시행 2024. 8. 14.', 'revise_info': '법률 제20305호, 2024. 2. 13., 일부개정', 'source' : '국가법령정보센터', 'title_doc': '제3편 증권의 발행 및 유통', 'title_chapter' : '제1장 증권신고서', 'title_section' : '제2절 금융투자업자별 영업행위 규칙', 'title_subsection' : '제4관 신탁업자의 영업행위 규칙'}}, 
{{'relation_type' : 'INCLUDE', 'description' : '증권신고서는 정정신고서를 포함함'}},
{{'entity_type' : 'OBJECT', 'law_index': '제125조제1항', 'name': '정정신고서', 'document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )', 'created_date': '시행 2024. 8. 14.', 'revise_info': '법률 제20305호, 2024. 2. 13., 일부개정', 'source' : '국가법령정보센터', 'title_doc': '제3편 증권의 발행 및 유통', 'title_chapter' : '제1장 증권신고서', 'title_section' : '제2절 금융투자업자별 영업행위 규칙', 'title_subsection' : '제4관 신탁업자의 영업행위 규칙'}}),

({{'entity_type' : 'OBJECT', 'law_index': '제125조제1항', 'name': '거짓의 기재 또는 표시', 'document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )', 'created_date': '시행 2024. 8. 14.', 'revise_info': '법률 제20305호, 2024. 2. 13., 일부개정', 'source' : '국가법령정보센터', 'title_doc': '제3편 증권의 발행 및 유통', 'title_chapter' : '제1장 증권신고서', 'title_section' : '제2절 금융투자업자별 영업행위 규칙', 'title_subsection' : '제4관 신탁업자의 영업행위 규칙'}}, 
{{'relation_type' : 'CAUSE', 'description' : '거짓의 기재 또는 표시는 배상책임을 야기한다'}},
{{'entity_type' : 'PENALTY', 'law_index': '제125조제1항', 'name': '배상책임', 'document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )', 'created_date': '시행 2024. 8. 14.', 'revise_info': '법률 제20305호, 2024. 2. 13., 일부개정', 'source' : '국가법령정보센터', 'title_doc': '제3편 증권의 발행 및 유통', 'title_chapter' : '제1장 증권신고서', 'title_section' : '제2절 금융투자업자별 영업행위 규칙', 'title_subsection' : '제4관 신탁업자의 영업행위 규칙'}})
]

<input>\n
{text}

<output>\n
"""

In [11]:
#  Metadata: law_index=>제44조제2항::name=>이해상충의 관리::document_title=>자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )
# ::created_date=>시행 2024. 8. 14.::revise_info=>법률 제20305호, 2024. 2. 13., 일부개정
# ::source=>국가법령정보센터::title_doc=>제2편 금융투자업::title_chapter=>제4장 영업행위 규칙::title_section=>제1절 공통 영업행위 규칙::title_subsection=>제1관 신의성실의무 등
# -----
# Text:
# 금융투자업자는 제1항에 따라 이해상충이 발생할 가능성을 파악ㆍ평가한 결과 이해상충이 발생할 가능성이 있다고 인정되는 경우에는 그 사실을 미리 해당 투자자에게 알려야 하며, 그 이해상충이 발생할 가능성을 내부통제기준이 정하는 방법 및 절차에 따라 투자자 보호에 문제가 없는 수준으로 낮춘 후 매매, 그 밖의 거래를 하여야 한다.\n
# Triplet:
# [
# ({{'entity_type' : 'LAW', 'law_index': '제44조제2항', 'name': '이해상충의 관리', 'document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )', 'created_date': '시행 2024. 8. 14.', 'revise_info': '법률 제20305호, 2024. 2. 13., 일부개정', 'source' : '국가법령정보센터', 'title_doc': '제2편 금융투자업', 'title_chapter' : '제4장 영업행위 규칙', 'title_section' : '제1절 공통 영업행위 규칙', 'title_subsection' : '제1관 신의성실의무 등'}}, 
# {{'relation_type' : 'REFERS_TO', 'description' : '동일한 조 제1항을 참조함'}},
# {{'entity_type' : 'LAW', 'law_index': '제44조제1항', 'name': '이해상충의 관리', 'document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )', 'created_date': '시행 2024. 8. 14.', 'revise_info': '법률 제20305호, 2024. 2. 13., 일부개정', 'source' : '국가법령정보센터', 'title_doc': '제2편 금융투자업', 'title_chapter' : '제4장 영업행위 규칙', 'title_section' : '제1절 공통 영업행위 규칙', 'title_subsection' : '제1관 신의성실의무 등'}}),

# ({{'entity_type' : 'HUMAN', 'law_index': '제44조제2항', 'name': '금융투자업자', 'document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )', 'created_date': '시행 2024. 8. 14.', 'revise_info': '법률 제20305호, 2024. 2. 13., 일부개정', 'source' : '국가법령정보센터', 'title_doc': '제2편 금융투자업', 'title_chapter' : '제4장 영업행위 규칙', 'title_section' : '제1절 공통 영업행위 규칙', 'title_subsection' : '제1관 신의성실의무 등'}}, 
# {{'relation_type' : 'DECIDE_OF', 'description' : '금융투자업자가 이해상충이 발생할 가능성을 파악'}},
# {{'entity_type' : 'OBJECT', 'law_index': '제44조제2항', 'name': '이해상충 발생 가능성', 'document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )', 'created_date': '시행 2024. 8. 14.', 'revise_info': '법률 제20305호, 2024. 2. 13., 일부개정', 'source' : '국가법령정보센터', 'title_doc': '제2편 금융투자업', 'title_chapter' : '제4장 영업행위 규칙', 'title_section' : '제1절 공통 영업행위 규칙', 'title_subsection' : '제1관 신의성실의무 등'}}),

# ({{'entity_type' : 'OBJECT', 'law_index': '제44조제2항', 'name': '내부통제기준', 'document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )', 'created_date': '시행 2024. 8. 14.', 'revise_info': '법률 제20305호, 2024. 2. 13., 일부개정', 'source' : '국가법령정보센터', 'title_doc': '제2편 금융투자업', 'title_chapter' : '제4장 영업행위 규칙', 'title_section' : '제1절 공통 영업행위 규칙', 'title_subsection' : '제1관 신의성실의무 등'}}, 
# {{'relation_type' : 'EFFECT', 'description' : '내부통제기준이 이해상충 발생 가능성에 영향을 미친다'}},
# {{'entity_type' : 'OBJECT', 'law_index': '제44조제2항', 'name': '이해상충 발생 가능성', 'document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )', 'created_date': '시행 2024. 8. 14.', 'revise_info': '법률 제20305호, 2024. 2. 13., 일부개정', 'source' : '국가법령정보센터', 'title_doc': '제2편 금융투자업', 'title_chapter' : '제4장 영업행위 규칙', 'title_section' : '제1절 공통 영업행위 규칙', 'title_subsection' : '제1관 신의성실의무 등'}}),
# ]

### SchemaLLMPathExtractor

In [18]:
from typing import Literal
from llama_index.core.indices.property_graph import DynamicLLMPathExtractor, SchemaLLMPathExtractor


entities = Literal["LAW", "HUMAN", "ACTION", "PENALTY", "OBJECT"]
relations = Literal["CONNECT", "DECIDE_OF", "ACT", "REFERS_TO", "CAUSE", "EQUALS_TO", "INCLUDE", "EFFECT"]
entity_props = list(documents[0].metadata.keys())
relation_props = ["relation_type", "description"]

# 가능한 triplet 형태 모두 정의
schema = {
    "LAW" : ["CONNECT", "REFERS_TO", "EQUALS_TO", "INCLUDE", "EFFECT"], # 관계, 노드 추가 가능
    "HUMAN" : ["CONNECT", "DECIDE_OF", "ACT", "REFERS_TO", "CAUSE", "EQUALS_TO", "INCLUDE", "EFFECT"],
    "ACTION" : ["CONNECT", "REFERS_TO", "CAUSE", "EQUALS_TO", "INCLUDE", "EFFECT"],
    "PENALTY" : ["CONNECT", "REFERS_TO", "CAUSE", "EQUALS_TO", "INCLUDE", "EFFECT"],
    "OBJECT" : ["CONNECT", "DECIDE_OF", "ACT", "REFERS_TO", "CAUSE", "EQUALS_TO", "INCLUDE", "EFFECT"]
    
}
# schema = [("LAW", "REFERS_TO", "LAW")]
# llm = OpenAI(temperature=0, model="gpt-4o")

In [19]:
# Extract graph from documents
import nest_asyncio 
nest_asyncio.apply()

schema_extractor=SchemaLLMPathExtractor(
            llm=Settings.llm,
            extract_prompt=triplet_prompt,
            max_triplets_per_chunk=3,
            possible_entities=entities,
            possible_relations=relations,
            possible_entity_props=entity_props,
            possible_relation_props=relation_props,
            kg_validation_schema=schema,
            strict=True, # if false, will allow triples outside of the schema
        )

In [ ]:
schema_index = PropertyGraphIndex.from_documents(
    documents,
    # embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    embed_kg_nodes=False,
    kg_extractors=[schema_extractor],
    property_graph_store=graph_store,
    show_progress=True,
)

In [ ]:
# Define retriever
retriever = schema_index.as_retriever(
)

results = retriever.retrieve("금융투자업자가 경영건전성기준을 충족하지 못하면 어떤 조치를 받나요?") 
type(results[0])
for record in results:
    print(record.text)
    print(record.metadata)

### DynamicLLMPathExtractor

> https://github.com/run-llama/llama_index/blob/ce43f766890234d066c339c915ebaa66ff292cc9/llama-index-core/llama_index/core/indices/property_graph/transformations/dynamic_llm.py#L162

In [ ]:
dynamic_extractor=DynamicLLMPathExtractor(
            extract_prompt=triplet_prompt,
            max_triplets_per_chunk=3,
            allowed_entity_types=entities,
            allowed_relation_types=relations,
            allowed_entity_props=entity_props,
            allowed_relation_props=relation_props,
            show_progress=True
        )

In [ ]:
dynamic_index = PropertyGraphIndex.from_documents(
    documents,
    # embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    embed_kg_nodes=False,
    kg_extractors=[schema_extractor],
    property_graph_store=graph_store,
    show_progress=True,
)

In [ ]:
# Define retriever
retriever = dynamic_index.as_retriever(
)

results = retriever.retrieve("금융투자업자가 경영건정성기준을 충족하지 못하면 어떤 조치를 받나요?")
for record in results:
    print(record.text)

## neo4j test

## (Optional)Build the Knowledge Graph with LlamaIndex

With the help of Llama Index and LLM defined, we could build Knowledge Graph from given documents.

If we have a Knowledge Graph on NebulaGraphStore already, this step could be skipped

In [7]:
import json
from llama_index.core import Document

# JSON 파일을 로드하는 함수
def load_json_as_documents(input_file):
    # 모든 노드 텍스트 가져오기
    

    # # schema에 맞추어 json flatten
    # flattened_json = [flatten_json(item) for item in json_data]

    documents = []
    
    # JSON 리스트의 각 요소를 Document로 변환하여 리스트에 추가
    for entry in json_data:
        # 각 요소를 Document 객체로 변환
        doc = Document(
            text=entry["content"],
            metadata={
                "law_index": entry["index"],
                "name": entry["subtitle"],
                "document_title": entry["metadata"]["document_title"],
                "created_date": entry["metadata"]["date"],
                "revise_info": entry["metadata"]["revise_info"],
                "source": entry["metadata"]["source"],
                "title_doc": entry["metadata"]["title"]["doc"],
                "title_chapter": entry["metadata"]["title"]["chapter"],
                "title_section": entry["metadata"]["title"]["section"],
                "title_subsection": entry["metadata"]["title"]["subsection"],
            },
            metadata_seperator="::",
            metadata_template="{key}=>{value}",
            text_template="Metadata: {metadata_str}\n-----\nContent: {content}",
        )
        documents.append(doc)
    
    return documents

# JSON 파일 경로
input_file = "/mnt/c/Users/Shic/legal_graph/results/1-2/DCM_1-2_nebula_test.json"

# 로더 사용하여 데이터 불러오기
documents = load_json_as_documents(input_file)


In [ ]:
# len(documents)

In [ ]:
# list(documents[0].metadata.keys())

테스트용 데이터 범위 제1조 ~ 제49조에서 제30조 ~ 제49조로 줄임 (Documents 188 -> 70)

In [ ]:
# from llama_index.core.schema import MetadataMode
# print(
#     "The LLM sees this: \n",
#     documents[1].get_content(metadata_mode=MetadataMode.LLM),
# )

In [ ]:
# print(
#     "The Embedding model sees this: \n",
#     documents[1].get_content(metadata_mode=MetadataMode.EMBED),
# )

### Step 2, Generate a KnowledgeGraphIndex with NebulaGraph as graph_store

Then, we will create a KnowledgeGraphIndex to enable Graph based RAG, see [here](https://gpt-index.readthedocs.io/en/latest/examples/index_structs/knowledge_graph/KnowledgeGraphIndex_vs_VectorStoreIndex_vs_CustomIndex_combined.html) for deails, apart from that, we have a Knowledge Graph up and running for other purposes, too!

> https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_documents/

In [12]:
# from llama_index.core import Prompt

triplet_prompt = """
다음과 같은 법률 데이터베이스가 구축되어 있습니다.

데이터베이스 생성 구문:
CREATE TAG law(law_index string, subtitle string, document_title string, created_date string, revise_info string, source string, title_doc string, title_chapter string, title_section string, title_subsection string);
CREATE EDGE REFERS_TO();
CREATE TAG INDEX law_index ON law(law_index(256));

데이터베이스에 노드와 엣지를 삽입하기 위해서, 주어진 텍스트에서 가능한 모든 사실과 법 조항 간의 관계를 추출하세요.
- 각 사실은 (주어, 관계, 객체)의 형태로 나타냅니다.
- 특히, 법 조항 간의 참조, 의무, 금지 등의 관계를 중점적으로 추출하세요.

예시:
 Metadata: law_index=>제44조제2항::subtitle=>이해상충의 관리::document_title=>자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )
::created_date=>시행 2024. 8. 14.::revise_info=>법률 제20305호, 2024. 2. 13., 일부개정
::source=>국가법령정보센터::title_doc=>제2편 금융투자업::title_chapter=>제4장 영업행위 규칙::title_section=>제1절 공통 영업행위 규칙::title_subsection=>제1관 신의성실의무 등
-----
Text: 금융투자업자는 제1항에 따라 이해상충이 발생할 가능성을 파악ㆍ평가한 결과 이해상충이 발생할 가능성이 있다고 인정되는 경우에는 그 사실을 미리 해당 투자자에게 알려야 하며, 그 이해상충이 발생할 가능성을 내부통제기준이 정하는 방법 및 절차에 따라 투자자 보호에 문제가 없는 수준으로 낮춘 후 매매, 그 밖의 거래를 하여야 한다.

삼중항:
(제44조제2항, 참조한다, 제44조제1항)
(금융투자업자, 파악ㆍ평가한다, 이해상충 발생 가능성)

텍스트: {text}
삼중항:
"""

In [ ]:
!pip install llama-index-prompts llama-index-llms

In [13]:
from llama_index.core import PromptTemplate  # 수정된 부분

triplet_template = PromptTemplate(template=triplet_prompt)  # 수정된 부분

In [ ]:
from llama_index.core import KnowledgeGraphIndex

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    # kg_triplet_extract_template 으로 triplet 추출 프롬프트 설정 가능
    kg_triplet_extract_template = triplet_template,
    storage_context=storage_context,
    max_triplets_per_chunk=5, # 범위 수정
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    # include_embeddings=True,
    show_progress=True
)

Now we have a Knowledge Graph on NebulaGraph cluster under space named `llamaindex` about the 'Guardians of the Galaxy Vol. 3' movie, let's play with it a little bit.

In [ ]:
# install related packages, password is nebula by default
%pip install ipython-ngql networkx pyvis jupyter_nebulagraph


In [ ]:
%load_ext ngql
%ngql --address 192.168.176.1 --port 9669 --user root --password 001101

In [ ]:
# Query some random Relationships with Cypher
%ngql USE llamaindex;
%ngql MATCH ()-[e]->() RETURN e LIMIT 20;

In [ ]:
%ngql MATCH ()-[e]->() RETURN DISTINCT e;

In [ ]:
%ngql MATCH ()-[e]->() RETURN DISTINCT e.relation;


In [ ]:
%ngql help

In [ ]:
%ngql USE llamaindex
%ng_draw

## Asking the Knowledge Graph

Finally, let's demo how to Query Knowledge Graph with Natural language!

Here, we will leverage the `KnowledgeGraphQueryEngine`, with `NebulaGraphStore` as the `storage_context.graph_store`.

In [1]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(temperature=0, model="gpt-4o")

In [ ]:
from llama_index.core.query_engine import KnowledgeGraphQueryEngine

from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore

query_engine = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    llm=llm,
    verbose=True,
)

In [ ]:
response = query_engine.generate_query(
    "증권신고서와 투자설명서의 부실기재에 대한 과징금이 부과되는 경우 부과대상과 부과금액은?",
)
display(Markdown(f"<b>{response}</b>"))

In [ ]:
graph_query = query_engine.generate_query(
    "Tell me about Peter Quill?",
)

graph_query = graph_query.replace("WHERE", "\n  WHERE").replace(
    "RETURN", "\nRETURN"
)

display(
    Markdown(
        f"""
```cypher
{graph_query}
```
"""
    )
)

We could see it helps generate the Graph query:

```cypher
MATCH (p:`entity`)-[:relationship]->(e:`entity`)
  WHERE p.`entity`.`name` == 'Peter Quill'
RETURN e.`entity`.`name`;
```
And synthese the question based on its result:

```json
{'e2.entity.name': ['grandfather', 'alternate version of Gamora', 'Guardians of the Galaxy']}
```

Of course we still could query it, too! And this query engine could be our best Graph Query Language learning bot, then :).

In [ ]:
%%ngql
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'Peter Quill'
RETURN p.`entity`.`name`, e.relationship, m.`entity`.`name`;

And change the query to be rendered

In [ ]:
%%ngql
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'Peter Quill'
RETURN p, e, m;

In [ ]:
%ng_draw

The results of this knowledge-fetching query could not be more clear from the renderred graph then.